# Effect of repealing charitable deduction

This identifies beneficiaries of the charitable deduction by modeling its repeal. Both repeal from current (2017) state and TCJA state are considered on a static basis. Change to after-tax income by decile and share of after-tax income held by top 10% are calculated.

*taxcalc version: 0.15.2  |  Data: CPS  |  Tax year: 2018  |  Type: Static  |  Author: Max Ghenis  |  Date run: 2018-01-31*

## Setup

### Imports

In [1]:
import taxcalc as tc
import pandas as pd
import numpy as np
import copy
from bokeh.io import show, output_notebook
import urllib as url_lib  # On Python 3.6 use "import urllib.request as url_lib".

In [2]:
tc.__version__

'0.16.0'

### Load reforms

Load from Github.

In [6]:
# Folders where reforms live.
GITHUB_BASE_URL = 'https://raw.githubusercontent.com/'

TAXCALC_GITHUB_BASE_URL = (GITHUB_BASE_URL +
                           'open-source-economics/Tax-Calculator/master/' +
                           'taxcalc/reforms/')

def read_url(url):
    return url_lib.urlopen(url).read()

def read_reform_taxcalc_github(reform_name):
    return read_url(TAXCALC_GITHUB_BASE_URL + reform_name + '.json')

def policy_from_reform(reform):
    pol = tc.Policy()
    pol.implement_reform(reform['policy'])
    if pol.reform_errors:
        print(pol.reform_errors)
    return pol

def create_static_policy_taxcalc_github(reform_name):
    reform = tc.Calculator.read_json_param_objects(
        read_reform_taxcalc_github(reform_name), None)
    return policy_from_reform(reform)

In [7]:
y2017_policy = create_static_policy_taxcalc_github(
    '2017_law')

In [8]:
no_charitable_reform = {2018: {'_ID_Charity_crt_all': [0]}}
y2017_no_charitable_policy = copy.deepcopy(y2017_policy)
y2017_no_charitable_policy.implement_reform(no_charitable_reform)
baseline_no_charitable_policy = tc.Policy()  # Can't combine with next step.
baseline_no_charitable_policy.implement_reform(no_charitable_reform)

### Specify `Calculator` objects for static analyses

In [9]:
recs = tc.Records.cps_constructor()

In [10]:
def static_baseline_calc(year):
    calc = tc.Calculator(records=recs, policy=tc.Policy())
    calc.advance_to_year(year)
    calc.calc_all()
    return calc

In [11]:
def static_calc_from_policy(pol, year):
    calc = tc.Calculator(records=recs, policy=pol)
    calc.advance_to_year(year)
    calc.calc_all()
    # Needs more if adding behavior.
    return calc

In [12]:
baseline_calc = static_baseline_calc(2018)
baseline_no_charitable_calc = static_calc_from_policy(baseline_no_charitable_policy, 2018)
y2017_calc = static_calc_from_policy(y2017_policy, 2018)
y2017_no_charitable_calc = static_calc_from_policy(y2017_no_charitable_policy, 2018)

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


### Revenue-neutral UBI

Create calculators for replacing the charitable deduction with UBI under 2017 and current law.

Start by calculating the total UBI amount (same for everyone, regardless of age).

#### Change to aggregate individual income tax revenue

In [13]:
baseline_rev_change = (baseline_no_charitable_calc.weighted_total('iitax') - 
                       baseline_calc.weighted_total('iitax'))
print ('Current: ${:0.1f}B'.
       format((baseline_rev_change) / 1e9))

y2017_rev_change = (y2017_no_charitable_calc.weighted_total('iitax') - 
                    y2017_calc.weighted_total('iitax'))
print ('2017: ${:0.1f}B'.
       format((y2017_rev_change) / 1e9))

Current: $21.5B
2017: $42.2B


In [15]:
baseline_df = baseline_calc.dataframe(['s006', 'nu18', 'n1820', 'n21'])
total_population = np.sum(baseline_df['s006'] * 
                          (baseline_df['nu18'] + 
                           baseline_df['n1820'] + 
                           baseline_df['n21']))
baseline_ubi_amount = baseline_rev_change / total_population
y2017_ubi_amount = y2017_rev_change / total_population
print ('Repealing the charitable deduction could finance a UBI of ${:0.0f}'.
       format(baseline_ubi_amount) +
       ' under current law, or ${:0.0f}'.format(y2017_ubi_amount) +
       ' under 2017 law.')

Repealing the charitable deduction could finance a UBI of $66 under current law, or $129 under 2017 law.


Create `Calculator` for replacing charitable deduction with UBI.

In [16]:
def apply_ubi(existing_policy, ubi_amount):
    ubi_reform = {2018: {'_UBI_u18': [ubi_amount], 
                         '_UBI_1820': [ubi_amount], 
                         '_UBI_21': [ubi_amount],
                         '_UBI_ecrt': [1.0]}}
    result_policy = copy.deepcopy(existing_policy)
    result_policy.implement_reform(ubi_reform)
    return static_calc_from_policy(result_policy, 2018)

In [17]:
baseline_ubi_no_charitable_calc = apply_ubi(
    baseline_no_charitable_policy, baseline_ubi_amount)
y2017_ubi_no_charitable_calc = apply_ubi(
    y2017_no_charitable_policy, y2017_ubi_amount)

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


Verify revenue-neutrality. 

*Use `aftertax_income` instead of `ii_tax` because the latter excludes UBI.*

In [18]:
baseline_ubi_rev_change = (baseline_ubi_no_charitable_calc.weighted_total('aftertax_income') - 
                           baseline_calc.weighted_total('aftertax_income'))
y2017_ubi_rev_change = (y2017_ubi_no_charitable_calc.weighted_total('aftertax_income') - 
                        y2017_calc.weighted_total('aftertax_income'))
print ('Revenue change from replacing charitable deduction with UBI (should be zero): ' +
       '${:0.1f}B'.format((baseline_ubi_rev_change) / 1e9) +
       ' under current law and ' +
       '${:0.1f}B'.format((y2017_ubi_rev_change) / 1e9) +
       ' under 2017 law.')

Revenue change from replacing charitable deduction with UBI (should be zero): $-0.0B under current law and $0.0B under 2017 law.


### `DataFrames`

Create `DataFrames` to more flexibly analyze and remove negative income tax units, which distort the story for the bottom decile.

In [19]:
def calc_df_w_percentile(calc, remove_negatives=True):
    df = calc.dataframe(['s006', 'expanded_income', 
                         'aftertax_income']).sort_values(by=['expanded_income'])
    if remove_negatives:
        df = df.loc[df['expanded_income'] >= 0]
    df['expanded_income_percentile'] = 100 * df['s006'].cumsum() / df['s006'].sum()
    df['expanded_income_decile'] = np.ceil(df['expanded_income_percentile'] / 10).astype(int)
    # One record is getting set to 11 due to rounding.
    df.loc[df['expanded_income_decile'] == 11, 'expanded_income_decile'] = 10
    # Set as string for charting.
    df['expanded_income_decile_str'] = (
        (10 * (df['expanded_income_decile'] - 1)).map(str) + '-' + 
        (10 * df['expanded_income_decile']).map(str))
    df['total_aftertax_income'] = df['s006'] * df['aftertax_income']
    return df

In [20]:
# Use _nn convention to indicate no negatives.
# Current baseline.
baseline_df_nn = calc_df_w_percentile(baseline_calc)
# 2017 law.
y2017_df_nn = calc_df_w_percentile(y2017_calc)
# Baseline without charitable deduction.
baseline_no_charitable_df_nn = calc_df_w_percentile(baseline_no_charitable_calc)
# 2017 law without charitable deduction.
y2017_no_charitable_df_nn = calc_df_w_percentile(y2017_no_charitable_calc)
# Baseline replacing charitable deduction with UBI.
baseline_ubi_df_nn = calc_df_w_percentile(baseline_ubi_no_charitable_calc)
# 2017 law replacing charitable deduction with UBI.
y2017_ubi_df_nn = calc_df_w_percentile(y2017_ubi_no_charitable_calc)

## Analysis

### Difference tables by income decile

Ignore errors ([issue](https://github.com/open-source-economics/Tax-Calculator/issues/1799)).

In [21]:
baseline_diff_table = baseline_calc.difference_table(baseline_no_charitable_calc, tax_to_diff='iitax')
y2017_diff_table = y2017_calc.difference_table(y2017_no_charitable_calc, tax_to_diff='iitax')
baseline_ubi_diff_table = baseline_calc.difference_table(baseline_ubi_no_charitable_calc, tax_to_diff='iitax')
y2017_ubi_diff_table = y2017_calc.difference_table(y2017_ubi_no_charitable_calc, tax_to_diff='iitax')

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ValueError: "Buffer dtype mismatch, expected 'Python object' but got 'long'" in 'pandas._libs.lib.is_bool_array' ignored


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ValueError: "Buffer dtype mismatch, expected 'Python object' but got 'long'" in 'pandas._libs.lib.is_bool_array' ignored


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ValueError: "Buffer dtype mismatch, expected 'Python object' but got 'long'" in 'pandas._libs.lib.is_bool_array' ignored


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ValueError: "Buffer dtype mismatch, expected 'Python object' but got 'long'" in 'pandas._libs.lib.is_bool_array' ignored


In [22]:
baseline_diff_table

,count,tax_cut,perc_cut,tax_inc,perc_inc,mean,tot_change,share_of_change,perc_aftertax,pc_aftertaxinc
0,"16,989,409.57",0.00,0.00,"2,708.86",0.02,0.02,"291,278.55",0.00,0.00,-0.00
1,"16,989,385.49",0.00,0.00,"154,474.08",0.91,1.63,"27,748,595.05",0.13,0.01,-0.01
2,"16,988,538.18",0.00,0.00,"351,849.55",2.07,5.71,"97,079,825.35",0.45,0.02,-0.02
3,"16,989,916.01",0.00,0.00,"506,882.10",2.98,11.73,"199,315,503.75",0.93,0.04,-0.04
4,"16,989,964.20",0.00,0.00,"751,290.09",4.42,22.58,"383,664,258.43",1.78,0.06,-0.06
5,"16,989,298.81",0.00,0.00,"1,077,267.36",6.34,44.98,"764,220,549.90",3.55,0.09,-0.09
6,"16,988,352.73",0.00,0.00,"1,665,898.21",9.81,82.67,"1,404,441,340.36",6.53,0.13,-0.13
7,"16,990,524.00",0.00,0.00,"2,916,852.08",17.17,118.54,"2,014,109,414.49",9.36,0.15,-0.15
8,"16,989,387.25",0.00,0.00,"4,846,318.62",28.53,306.69,"5,210,501,554.06",24.22,0.28,-0.28
9,"16,990,063.42",0.00,0.00,"6,348,504.05",37.37,671.84,"11,414,563,555.21",53.05,0.30,-0.30


In [23]:
y2017_diff_table

,count,tax_cut,perc_cut,tax_inc,perc_inc,mean,tot_change,share_of_change,perc_aftertax,pc_aftertaxinc
0,"16,989,409.57",0.00,0.00,"12,084.91",0.07,0.06,"986,584.27",0.00,0.00,-0.00
1,"16,989,385.49",0.00,0.00,"759,714.80",4.47,6.58,"111,785,070.47",0.27,0.04,-0.04
2,"16,988,538.18",0.00,0.00,"1,873,618.35",11.03,27.79,"472,078,223.42",1.12,0.11,-0.11
3,"16,989,916.01",0.00,0.00,"2,380,224.64",14.01,52.78,"896,764,354.56",2.13,0.17,-0.17
4,"16,989,964.20",0.00,0.00,"2,912,171.71",17.14,88.05,"1,495,987,055.59",3.55,0.22,-0.22
5,"16,989,298.81",0.00,0.00,"3,619,123.82",21.30,158.11,"2,686,222,820.95",6.37,0.32,-0.32
6,"16,988,352.73",0.00,0.00,"4,225,072.70",24.87,254.77,"4,328,119,637.30",10.26,0.40,-0.40
7,"16,990,524.00",0.00,0.00,"5,009,768.29",29.49,339.43,"5,767,101,764.51",13.68,0.42,-0.42
8,"16,989,387.25",0.00,0.00,"6,292,840.05",37.04,556.01,"9,446,324,862.25",22.40,0.52,-0.52
9,"16,990,063.42",0.00,0.00,"7,560,054.16",44.50,998.59,"16,966,073,364.22",40.23,0.46,-0.46


UBI diff tables. % after-tax income for bottom decile is skewed due to that group having negative after-tax income in the baseline.

In [24]:
baseline_ubi_diff_table

,count,tax_cut,perc_cut,tax_inc,perc_inc,mean,tot_change,share_of_change,perc_aftertax,pc_aftertaxinc
0,"16,989,409.57",0.00,0.00,"2,708.86",0.02,0.02,"291,278.55",0.00,0.00,2.21
1,"16,989,385.49",0.00,0.00,"154,474.08",0.91,1.63,"27,748,595.05",0.13,0.01,0.51
2,"16,988,538.18",0.00,0.00,"351,849.55",2.07,5.71,"97,079,825.35",0.45,0.02,0.37
3,"16,989,916.01",0.00,0.00,"506,882.10",2.98,11.73,"199,315,503.75",0.93,0.04,0.30
4,"16,989,964.20",0.00,0.00,"751,290.09",4.42,22.58,"383,664,258.43",1.78,0.06,0.24
5,"16,989,298.81",0.00,0.00,"1,077,267.36",6.34,44.98,"764,220,549.90",3.55,0.09,0.17
6,"16,988,352.73",0.00,0.00,"1,665,898.21",9.81,82.67,"1,404,441,340.36",6.53,0.13,0.09
7,"16,990,524.00",0.00,0.00,"2,916,852.08",17.17,118.54,"2,014,109,414.49",9.36,0.15,0.04
8,"16,989,387.25",0.00,0.00,"4,846,318.62",28.53,306.69,"5,210,501,554.06",24.22,0.28,-0.13
9,"16,990,063.42",0.00,0.00,"6,348,504.05",37.37,671.84,"11,414,563,555.21",53.05,0.30,-0.22


In [25]:
y2017_ubi_diff_table

,count,tax_cut,perc_cut,tax_inc,perc_inc,mean,tot_change,share_of_change,perc_aftertax,pc_aftertaxinc
0,"16,989,409.57",0.00,0.00,"12,084.91",0.07,0.06,"986,584.27",0.00,0.00,4.34
1,"16,989,385.49",0.00,0.00,"759,714.80",4.47,6.58,"111,785,070.47",0.27,0.04,0.99
2,"16,988,538.18",0.00,0.00,"1,873,618.35",11.03,27.79,"472,078,223.42",1.12,0.11,0.67
3,"16,989,916.01",0.00,0.00,"2,380,224.64",14.01,52.78,"896,764,354.56",2.13,0.17,0.50
4,"16,989,964.20",0.00,0.00,"2,912,171.71",17.14,88.05,"1,495,987,055.59",3.55,0.22,0.37
5,"16,989,298.81",0.00,0.00,"3,619,123.82",21.30,158.11,"2,686,222,820.95",6.37,0.32,0.20
6,"16,988,352.73",0.00,0.00,"4,225,072.70",24.87,254.77,"4,328,119,637.30",10.26,0.40,0.03
7,"16,990,524.00",0.00,0.00,"5,009,768.29",29.49,339.43,"5,767,101,764.51",13.68,0.42,-0.05
8,"16,989,387.25",0.00,0.00,"6,292,840.05",37.04,556.01,"9,446,324,862.25",22.40,0.52,-0.21
9,"16,990,063.42",0.00,0.00,"7,560,054.16",44.50,998.59,"16,966,073,364.22",40.23,0.46,-0.30


### % change to after-tax income

Focus on % change to after-tax income. Build new `DataFrame`s to address portion of bottom decile with negative income.

In [26]:
def decile_after_tax_summary(calc):
    return calc.groupby('expanded_income_decile_str', 
                        as_index=False)['total_aftertax_income'].sum()

In [27]:
# Use _s to indicate summary.
baseline_df_nn_s = decile_after_tax_summary(baseline_df_nn)
y2017_df_nn_s = decile_after_tax_summary(y2017_df_nn)
baseline_no_charitable_df_nn_s = decile_after_tax_summary(baseline_no_charitable_df_nn)
y2017_no_charitable_df_nn_s = decile_after_tax_summary(y2017_no_charitable_df_nn)
baseline_ubi_df_nn_s = decile_after_tax_summary(baseline_ubi_df_nn)
y2017_ubi_df_nn_s = decile_after_tax_summary(y2017_ubi_df_nn)

In [28]:
def decile_comparison(base, comparison):
    decile_comparison = pd.merge(base, comparison, on='expanded_income_decile_str')
    decile_comparison['pct_change'] = (
        decile_comparison['total_aftertax_income_y'] / 
        decile_comparison['total_aftertax_income_x']) - 1
    return decile_comparison

In [29]:
baseline_ubi_diff = decile_comparison(baseline_df_nn_s, baseline_ubi_df_nn_s)
baseline_ubi_diff

,expanded_income_decile_str,total_aftertax_income_x,total_aftertax_income_y,pct_change
0,0-10,"93,652,313,734.82","95,038,103,156.55",0.01
1,10-20,"294,830,123,011.21","296,180,815,351.15",0.00
2,20-30,"426,093,032,417.68","427,613,796,006.84",0.00
3,30-40,"547,386,261,966.89","549,060,747,278.46",0.00
4,40-50,"687,346,229,450.57","688,832,968,345.66",0.00
5,50-60,"858,291,205,015.51","859,917,286,611.98",0.00
6,60-70,"1,080,144,072,305.04","1,081,257,088,161.72",0.00
7,70-80,"1,378,085,597,555.52","1,378,595,815,130.32",0.00
8,80-90,"1,852,284,702,147.77","1,850,062,152,808.48",-0.00
9,90-100,"3,758,392,625,300.55","3,749,924,048,061.45",-0.00


In [30]:
y2017_ubi_diff = decile_comparison(y2017_df_nn_s, y2017_ubi_df_nn_s)
y2017_ubi_diff

,expanded_income_decile_str,total_aftertax_income_x,total_aftertax_income_y,pct_change
0,0-10,"93,379,583,138.78","96,083,771,841.08",0.03
1,10-20,"293,109,599,887.48","295,770,635,048.66",0.01
2,20-30,"422,604,611,425.28","425,317,084,864.99",0.01
3,30-40,"541,810,104,195.05","544,554,225,936.52",0.01
4,40-50,"679,932,885,927.61","682,027,616,215.25",0.00
5,50-60,"849,346,557,640.26","851,433,765,464.31",0.00
6,60-70,"1,069,732,463,827.18","1,070,329,182,107.85",0.00
7,70-80,"1,363,032,532,023.88","1,362,152,836,107.74",-0.00
8,80-90,"1,833,072,817,555.94","1,829,369,120,508.14",-0.00
9,90-100,"3,694,051,842,095.75","3,682,989,063,851.08",-0.00


Since total incomes are nearly equal\*, the inequality statistic "share of after-tax income held by top decile" is straightforward: -0.26% in the baseline, -0.4% under 2017 law.

\* Tiny differences due to some records being removed if negative.